# Importing the necessary libraries

In [93]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import boto3
import urllib.request, json, os, sagemaker
from sagemaker import get_execution_role
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [94]:
pip install boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


# Creating buckets and loading dataset

In [95]:
import boto3

# Let's use Amazon S3
s3 = boto3.resource("s3")

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

auto-ml-test1
automl-test2
finalproject22a
nmv-pipeline
nmv-ver1
sagemaker-project-p-vxg2dy8ptpix
sagemaker-us-east-1-618402266652


In [96]:
s3 = boto3.resource('s3')

In [97]:
bucket_name = 'finalproject22a'

In [98]:
my_region = boto3.session.Session().region_name
prefix = 'sagemaker/MLI-DEMO-xgboost-dm'

print("Region: {}".format(my_region))

Region: us-east-1


In [99]:
try:
    if  my_region == 'us-east-1':
        if not s3.Bucket(bucket_name).creation_date:
            s3.create_bucket(Bucket=bucket_name)
            print('S3 bucket created successfully')
        else:
            print('Bucket already exists!')
    else: 
        if not s3.Bucket(bucket_name).creation_date:
            s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
            print('S3 bucket created successfully')
        else:
            print('Bucket already exists!')
except Exception as e:
    print('S3 error: ', e)

Bucket already exists!


In [100]:
import os

DATASET = 'loan_train.csv'
DATA_FOLDER = 's3://finalproject22a/archive'
data = pd.read_csv(os.path.join(DATA_FOLDER, DATASET))

# Preprocessing

In [101]:
data.isna().sum()

Gender                13
Married                3
Dependents            15
Education              0
Self_Employed         32
Applicant_Income       0
Coapplicant_Income     0
Loan_Amount            0
Term                  14
Credit_History        50
Area                   0
Status                 0
dtype: int64

In [102]:
loan = data.dropna()

In [103]:
# Convert the record_id field from an integer to a float
loan['Gender'] = loan['Gender'].astype('string')
loan['Married'] = loan['Married'].astype('string')
loan['Dependents'] = loan['Dependents'].astype('string')
loan['Education'] = loan['Education'].astype('string')
loan['Self_Employed'] = loan['Self_Employed'].astype('string')
loan['Area'] = loan['Area'].astype('string')
loan['Status'] = loan['Status'].astype('string')
loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 613
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              499 non-null    string 
 1   Married             499 non-null    string 
 2   Dependents          499 non-null    string 
 3   Education           499 non-null    string 
 4   Self_Employed       499 non-null    string 
 5   Applicant_Income    499 non-null    int64  
 6   Coapplicant_Income  499 non-null    float64
 7   Loan_Amount         499 non-null    int64  
 8   Term                499 non-null    float64
 9   Credit_History      499 non-null    float64
 10  Area                499 non-null    string 
 11  Status              499 non-null    string 
dtypes: float64(3), int64(2), string(7)
memory usage: 50.7 KB


/tmp/ipykernel_27079/3238559731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan['Gender'] = loan['Gender'].astype('string')
/tmp/ipykernel_27079/3238559731.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan['Married'] = loan['Married'].astype('string')
/tmp/ipykernel_27079/3238559731.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [104]:
from sklearn.preprocessing import LabelEncoder

for i in loan.columns:
    if loan[i].dtypes == 'string':
        loan[i] =  LabelEncoder().fit_transform(loan[i])

/tmp/ipykernel_27079/3933336101.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan[i] =  LabelEncoder().fit_transform(loan[i])


In [105]:
from sklearn.preprocessing import LabelEncoder

for i in loan.columns:
    if loan[i].dtypes == 'string':
        loan[i] =  LabelEncoder().fit_transform(loan[i])

In [106]:
loan2=loan.copy()

In [107]:
loan2

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
0,1,0,0,0,0,584900,0.0,15000000,360.0,1.0,2,1
1,1,1,1,0,0,458300,150800.0,12800000,360.0,1.0,0,0
2,1,1,0,0,1,300000,0.0,6600000,360.0,1.0,2,1
3,1,1,0,1,0,258300,235800.0,12000000,360.0,1.0,2,1
4,1,0,0,0,0,600000,0.0,14100000,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,290000,0.0,7100000,360.0,1.0,0,1
610,1,1,3,0,0,410600,0.0,4000000,180.0,1.0,0,1
611,1,1,1,0,0,807200,24000.0,25300000,360.0,1.0,2,1
612,1,1,2,0,0,758300,0.0,18700000,360.0,1.0,2,1


In [108]:
loan3 = loan2[loan2['Loan_Amount'] != 0] 
    
print('\nResult dataframe :\n', loan3)


Result dataframe :
      Gender  Married  Dependents  Education  Self_Employed  Applicant_Income  \
0         1        0           0          0              0            584900   
1         1        1           1          0              0            458300   
2         1        1           0          0              1            300000   
3         1        1           0          1              0            258300   
4         1        0           0          0              0            600000   
..      ...      ...         ...        ...            ...               ...   
609       0        0           0          0              0            290000   
610       1        1           3          0              0            410600   
611       1        1           1          0              0            807200   
612       1        1           2          0              0            758300   
613       0        0           0          0              1            458300   

     Coapplicant_I

In [109]:
loan3

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
0,1,0,0,0,0,584900,0.0,15000000,360.0,1.0,2,1
1,1,1,1,0,0,458300,150800.0,12800000,360.0,1.0,0,0
2,1,1,0,0,1,300000,0.0,6600000,360.0,1.0,2,1
3,1,1,0,1,0,258300,235800.0,12000000,360.0,1.0,2,1
4,1,0,0,0,0,600000,0.0,14100000,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,290000,0.0,7100000,360.0,1.0,0,1
610,1,1,3,0,0,410600,0.0,4000000,180.0,1.0,0,1
611,1,1,1,0,0,807200,24000.0,25300000,360.0,1.0,2,1
612,1,1,2,0,0,758300,0.0,18700000,360.0,1.0,2,1


In [110]:
loan3.isna().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
Applicant_Income      0
Coapplicant_Income    0
Loan_Amount           0
Term                  0
Credit_History        0
Area                  0
Status                0
dtype: int64

In [111]:
loan3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481 entries, 0 to 613
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              481 non-null    int64  
 1   Married             481 non-null    int64  
 2   Dependents          481 non-null    int64  
 3   Education           481 non-null    int64  
 4   Self_Employed       481 non-null    int64  
 5   Applicant_Income    481 non-null    int64  
 6   Coapplicant_Income  481 non-null    float64
 7   Loan_Amount         481 non-null    int64  
 8   Term                481 non-null    float64
 9   Credit_History      481 non-null    float64
 10  Area                481 non-null    int64  
 11  Status              481 non-null    int64  
dtypes: float64(3), int64(9)
memory usage: 48.9 KB


In [112]:
loan3.describe()

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
count,481.000000,481.000000,481.000000,481.000000,481.000000,4.810000e+02,4.810000e+02,4.810000e+02,481.000000,481.000000,481.000000,481.000000
mean,0.821206,0.646570,0.775468,0.201663,0.137214,5.365239e+05,1.577806e+05,1.447464e+07,342.087318,0.854470,1.024948,0.692308
std,0.383579,0.478533,1.020366,0.401660,0.344431,5.662387e+05,2.615958e+05,8.042462e+06,65.149577,0.353002,0.776880,0.462019
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+04,0.000000e+00,9.000000e+05,36.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,2.900000e+05,0.000000e+00,1.000000e+07,360.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,3.859000e+05,1.083000e+05,1.280000e+07,360.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,2.000000,0.000000,0.000000,5.849000e+05,2.253000e+05,1.700000e+07,360.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,3.000000,1.000000,1.000000,8.100000e+06,3.383700e+06,6.000000e+07,480.000000,1.000000,2.000000,1.000000


In [113]:
loan3.value_counts('Status')

Status
1    333
0    148
dtype: int64

In [114]:
loan3.corr()

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
Gender,1.000000,0.347364,0.216590,0.058727,-0.003147,0.032719,0.155489,0.099016,-0.088410,0.022839,0.001017,0.065108
Married,0.347364,1.000000,0.387542,0.003065,0.016766,0.036406,0.104416,0.182907,-0.108066,0.027871,0.034975,0.110176
Dependents,0.216590,0.387542,1.000000,0.029379,0.046351,0.130924,0.000639,0.172572,-0.096731,-0.027284,-0.000803,0.034334
Education,0.058727,0.003065,0.029379,1.000000,-0.004665,-0.131226,-0.073819,-0.172802,-0.102421,-0.057063,-0.056216,-0.069086
Self_Employed,-0.003147,0.016766,0.046351,-0.004665,1.000000,0.170684,-0.001006,0.120314,-0.035072,-0.023903,-0.051749,-0.035247
Applicant_Income,0.032719,0.036406,0.130924,-0.131226,0.170684,1.000000,-0.112652,0.495315,-0.010788,-0.056068,-0.052848,-0.043012
Coapplicant_Income,0.155489,0.104416,0.000639,-0.073819,-0.001006,-0.112652,1.000000,0.190584,-0.006118,-0.009206,0.004946,-0.049817
Loan_Amount,0.099016,0.182907,0.172572,-0.172802,0.120314,0.495315,0.190584,1.000000,0.050900,-0.040709,-0.109333,-0.071628
Term,-0.088410,-0.108066,-0.096731,-0.102421,-0.035072,-0.010788,-0.006118,0.050900,1.000000,0.033165,-0.057834,-0.007411
Credit_History,0.022839,0.027871,-0.027284,-0.057063,-0.023903,-0.056068,-0.009206,-0.040709,0.033165,1.000000,-0.001927,0.529624


# Spliting the dataset 

In [116]:
import boto3
import urllib.request, json, os, sagemaker
from sagemaker import get_execution_role
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [117]:
train_data, validation_data, test_data = np.split(
    loan3.sample(frac=1, random_state=1730),
    [int(0.7 * len(loan3)), int(0.9 * len(loan3))],
)


print(train_data.shape, test_data.shape, validation_data.shape)

(336, 12) (49, 12) (96, 12)


In [118]:
label_column = train_data['Status']
train_data = train_data.drop(['Status'], axis=1)
train_data = pd.concat([label_column, train_data], axis=1)

train_data.to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [119]:
train_data.head()

,Status,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
14,1,1,1,2,0,0,129900,108600.0,1700000,120.0,1.0,2
342,1,1,1,1,1,0,455400,122900.0,15800000,360.0,1.0,2
418,1,1,1,0,0,0,102500,277300.0,11200000,360.0,1.0,0
328,0,0,1,0,0,0,433300,245100.0,11000000,360.0,1.0,2
593,1,1,1,0,0,0,385900,330000.0,14200000,180.0,1.0,0


In [120]:
label_column = validation_data['Status']
validation_data = validation_data.drop(['Status'], axis=1)
validation_data = pd.concat([label_column, validation_data], axis=1)

validation_data.to_csv('validation.csv', index=False, header=False)

boto3.Session().resource("s3").Bucket(bucket_name).Object(os.path.join(prefix, "validation/validation.csv")).upload_file("validation.csv")

s3_input_validation = sagemaker.TrainingInput(s3_data='s3://{}/{}/validation'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [121]:
validation_data.head()

,Status,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
141,1,1,0,0,0,0,541700,0.0,16800000,360.0,1.0,2
502,1,1,1,2,0,0,486500,562400.0,20800000,360.0,1.0,1
584,0,1,1,1,0,0,278700,191700.0,14600000,360.0,0.0,0
456,1,1,1,0,0,0,430100,0.0,11800000,360.0,1.0,2
462,1,1,1,0,0,0,301500,218800.0,15300000,360.0,1.0,0


# Initiating sagemaker xgboost and model creation

In [122]:
sess = sagemaker.Session()

In [123]:
role = get_execution_role()

In [124]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [125]:
xgb = sagemaker.estimator.Estimator(xgboost_container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [126]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

# Train the dataset

In [127]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-04-13-15-16-01-842


2023-04-13 15:16:02 Starting - Starting the training job...
2023-04-13 15:16:28 Starting - Preparing the instances for training......
2023-04-13 15:17:28 Downloading - Downloading input data...
2023-04-13 15:17:53 Training - Downloading the training image.....Arguments: train
[2023-04-13:15:18:44:INFO] Running standalone xgboost training.
[2023-04-13:15:18:44:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-04-13:15:18:44:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8610.31mb
[2023-04-13:15:18:44:INFO] Determined delimiter of CSV input is ','
[15:18:44] S3DistributionType set as FullyReplicated
[15:18:44] 336x11 matrix with 3696 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[15:18:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 12 pruned nodes, max_depth=1
[0]#011train-error:0.208333
[15:18:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra no

In [128]:
objective_metric_name = "validation:auc"
MAX_JOBS = 5
MAX_PARALLEL_JOBS = 4
STRATEGY = 'Bayesian'
SCALING_TYPE = 'Linear'

In [129]:
tuninig_job_name = "xgb-linsearch-" + strftime("%Y%m%d-%H-%M-%S", gmtime())

hyperparameter_ranges_linear = {
    "alpha": ContinuousParameter(0.05, 1, scaling_type=SCALING_TYPE),
    "lambda": ContinuousParameter(0.05, 1, scaling_type=SCALING_TYPE),
}

tuner_linear = HyperparameterTuner(
    xgb,
    objective_metric_name,
    hyperparameter_ranges_linear,
    max_jobs=MAX_JOBS,
    max_parallel_jobs=MAX_PARALLEL_JOBS,
    strategy=STRATEGY,
)

tuner_linear.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
    include_cls_metadata=False,
    job_name=tuninig_job_name)

INFO:sagemaker:Creating hyperparameter tuning job with name: xgb-linsearch-20230413-15-19-23


..................................................................!


In [130]:
boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_linear.latest_tuning_job.job_name
)["HyperParameterTuningJobStatus"]

'Completed'

In [131]:
tuner = sagemaker.HyperparameterTuningJobAnalytics(tuninig_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=False)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 5
{'lowest': 0.7246379852294922, 'highest': 0.7842190265655518}


,alpha,lambda,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
2,0.555397,0.795821,xgb-linsearch-20230413-15-19-23-003-38107cc6,Completed,0.784219,2023-04-13 15:21:19+00:00,2023-04-13 15:23:16+00:00,117.0
1,0.846494,0.278512,xgb-linsearch-20230413-15-19-23-004-68bf6972,Completed,0.747182,2023-04-13 15:21:31+00:00,2023-04-13 15:23:17+00:00,106.0
4,0.252974,0.462469,xgb-linsearch-20230413-15-19-23-001-cc8b16ff,Completed,0.747182,2023-04-13 15:21:17+00:00,2023-04-13 15:22:59+00:00,102.0
0,0.316903,0.238946,xgb-linsearch-20230413-15-19-23-005-8f7ec569,Completed,0.741278,2023-04-13 15:23:27+00:00,2023-04-13 15:24:46+00:00,79.0
3,0.366163,0.078629,xgb-linsearch-20230413-15-19-23-002-edb714bd,Completed,0.724638,2023-04-13 15:21:19+00:00,2023-04-13 15:22:52+00:00,93.0


# Deploy the model and creating endpoint

In [133]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-04-13-15-25-19-248
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-04-13-15-25-19-248
INFO:sagemaker:Creating endpoint with name xgboost-2023-04-13-15-25-19-248


-------!

In [134]:
test_data_array = test_data.drop(['Status'], axis=1).values

In [135]:
xgb_predictor.serializer = csv_serializer

In [136]:
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(49,)


In [137]:
predictions_array

array([0.76343787, 0.46597332, 0.70553535, 0.83843571, 0.82133377,
       0.90497601, 0.72584224, 0.23788796, 0.78886515, 0.65653336,
       0.72160769, 0.76274806, 0.79909384, 0.74878019, 0.69548339,
       0.87321091, 0.87321091, 0.19923036, 0.78186738, 0.83322304,
       0.17902289, 0.19555779, 0.81155813, 0.23835607, 0.88869292,
       0.85579586, 0.87321091, 0.14648503, 0.69306409, 0.19666469,
       0.87843382, 0.81664765, 0.82935792, 0.71457011, 0.55616462,
       0.8456676 , 0.817568  , 0.80307311, 0.92997241, 0.71048701,
       0.58344918, 0.79251015, 0.82907891, 0.64178544, 0.90586245,
       0.13079849, 0.20029396, 0.66306961, 0.82758921])

In [138]:
from sklearn.metrics import classification_report
print(classification_report(test_data['Status'], np.round(predictions_array), target_names=['approve', 'deny']))

              precision    recall  f1-score   support

     approve       0.90      0.56      0.69        16
        deny       0.82      0.97      0.89        33

    accuracy                           0.84        49
   macro avg       0.86      0.77      0.79        49
weighted avg       0.85      0.84      0.82        49



In [139]:
from sklearn import metrics

In [140]:
auc = metrics.roc_auc_score(test_data['Status'], predictions_array)

In [141]:
auc

0.7481060606060606

# Test the performance of the model

In [143]:
endpoint_name = 'xgboost-2023-04-13-15-25-19-248'

In [144]:
from sagemaker.predictor import Predictor

In [145]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name,
    sagemaker_session=sess,
)

In [185]:
sample_record_csv = '''Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
1,0,0,1,0,584900,0,15000000,360,1,2
'''

In [186]:
import pandas as pd
from io import StringIO

In [187]:
sample_record_io = StringIO(sample_record_csv)
sample_df = pd.read_csv(sample_record_io)

In [188]:
sample_record_io = StringIO(sample_record_csv)
sample_df = pd.read_csv(sample_record_io)

#if contains_target_column:
    # Drop the target column from the sample csv
    #print(f"Target column value of sample record: {sample_df.iloc[0][target_column_name]}")
    #sample_df = sample_df.drop(columns=[target_column_name])

sample_record_payload = sample_df.to_csv(header=False, index=False)
print(f"Sample record to predict: {sample_record_payload}")


Sample record to predict: 1,0,0,1,0,584900,0,15000000,360,1,2



In [189]:
prediction = predictor.predict(sample_record_payload, initial_args={"ContentType": "text/csv"})

print(f"The predicted target is: {prediction}")
if prediction > b'0.7':
  print("Loan has been approved")
elif prediction < b'0.7':
  print("Loan has been denied")

The predicted target is: b'0.8253770470619202'
Loan has been approved


In [190]:
sample_record_csv2 = '''Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
0,1,0,1,0,90900,0,1000000,360,0,2
'''

In [191]:
sample_record_io = StringIO(sample_record_csv2)
sample_df = pd.read_csv(sample_record_io)

In [192]:
sample_record_io = StringIO(sample_record_csv2)
sample_df = pd.read_csv(sample_record_io)

#if contains_target_column:
    # Drop the target column from the sample csv
    #print(f"Target column value of sample record: {sample_df.iloc[0][target_column_name]}")
    #sample_df = sample_df.drop(columns=[target_column_name])

sample_record_payload = sample_df.to_csv(header=False, index=False)
print(f"Sample record to predict: {sample_record_payload}")

Sample record to predict: 0,1,0,1,0,90900,0,1000000,360,0,2



In [193]:
prediction = predictor.predict(sample_record_payload, initial_args={"ContentType": "text/csv"})

print(f"The predicted target is: {prediction}")
if prediction > b'0.7':
  print("Loan has been approved")
elif prediction < b'0.7':
  print("Loan has been denied")

The predicted target is: b'0.09601276367902756'
Loan has been denied


In [194]:
sample_record_csv3 = '''Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
0,1,1,1,0,900,0,5000000,360,1,1
'''

In [195]:
sample_record_io = StringIO(sample_record_csv3)
sample_df = pd.read_csv(sample_record_io)

#if contains_target_column:
    # Drop the target column from the sample csv
    #print(f"Target column value of sample record: {sample_df.iloc[0][target_column_name]}")
    #sample_df = sample_df.drop(columns=[target_column_name])

sample_record_payload = sample_df.to_csv(header=False, index=False)
print(f"Sample record to predict: {sample_record_payload}")

Sample record to predict: 0,1,1,1,0,900,0,5000000,360,1,1



In [196]:
prediction = predictor.predict(sample_record_payload, initial_args={"ContentType": "text/csv"})

print(f"The predicted target is: {prediction}")
if prediction > b'0.7':
  print("Loan has been approved")
elif prediction < b'0.7':
  print("Loan has been denied")

The predicted target is: b'0.5597441792488098'
Loan has been denied


# Deleting the endpoint

In [111]:
sess.delete_endpoint(endpoint_name=xgb_predictor.endpoint_name)
#bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
#bucket_to_delete.objects.all().delete()

INFO:sagemaker:Deleting endpoint with name: xgboost-2023-04-10-19-17-45-819


[{'ResponseMetadata': {'RequestId': '0KK5Q1P1E8052R10',
   'HostId': 'x8a1fHcQpA4ErVcZ4nZSdEsg9Yq/yBNhi2yZ6YH/OEtYTpsRHCToqZ33Y2ySf6FRRqCxO8TQVHg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'x8a1fHcQpA4ErVcZ4nZSdEsg9Yq/yBNhi2yZ6YH/OEtYTpsRHCToqZ33Y2ySf6FRRqCxO8TQVHg=',
    'x-amz-request-id': '0KK5Q1P1E8052R10',
    'date': 'Mon, 10 Apr 2023 19:21:59 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/MLI-DEMO-xgboost-dm/output/xgb-linsearch-20230410-18-58-02-002-bb332d7e/output/model.tar.gz',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'BDpnosAt__qxcZ.KebB4lfH_B8bXPJfx'},
   {'Key': 'sagemaker/MLI-DEMO-xgboost-dm/output/xgboost-2023-04-10-18-53-15-484/profiler-output/system/training_job_end.ts',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'F_ftEfKg9sdA7.Hc6RNOjpLp6dzWt_eW'},
   {'Key': 'archive/loan_train.cs